# Grid-Based Blind Search Algorithms with Real-Time Visualization
## Six fundamental search strategies: BFS, DFS, UCS, DLS, IDDFS, Bidirectional Search

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle
from collections import deque
import heapq
import time
from IPython.display import display, clear_output
import copy

## Grid Configuration and Helper Functions

In [ ]:
class GridWorld:
    def __init__(self, rows=15, cols=15):
        self.rows = rows
        self.cols = cols
        self.grid = np.zeros((rows, cols), dtype=int)
        
        # Cell types
        self.EMPTY = 0
        self.WALL = 1
        self.START = 2
        self.TARGET = 3
        self.VISITED = 4
        self.PATH = 5
        self.FRONTIER = 6
        
        # Set start and target
        self.start = (1, 1)
        self.target = (13, 13)
        
        # Initialize grid
        self.reset_grid()
        
    def reset_grid(self):
        """Reset the grid to initial state"""
        self.grid = np.zeros((self.rows, self.cols), dtype=int)
        
        # Add some walls to make it interesting
        # Vertical walls
        for i in range(5, 12):
            self.grid[i, 5] = self.WALL
            self.grid[i, 10] = self.WALL
        
        # Horizontal walls
        for j in range(3, 8):
            self.grid[7, j] = self.WALL
            
        for j in range(8, 13):
            self.grid[10, j] = self.WALL
        
        # Set start and target
        self.grid[self.start[0], self.start[1]] = self.START
        self.grid[self.target[0], self.target[1]] = self.TARGET
    
    def get_neighbors(self, pos):
        """
        Get valid neighbors in strict clockwise order:
        1. Up
        2. Right
        3. Bottom
        4. Bottom-Right (Diagonal)
        5. Left
        6. Top-Left (Diagonal)
        """
        row, col = pos
        neighbors = []
        
        # Define moves in the required order
        moves = [
            (-1, 0),   # Up
            (0, 1),    # Right
            (1, 0),    # Bottom
            (1, 1),    # Bottom-Right (Diagonal)
            (0, -1),   # Left
            (-1, -1)   # Top-Left (Diagonal)
        ]
        
        for dr, dc in moves:
            new_row, new_col = row + dr, col + dc
            
            # Check if the position is valid
            if (0 <= new_row < self.rows and 
                0 <= new_col < self.cols and 
                self.grid[new_row, new_col] != self.WALL):
                
                # For diagonal moves, check that both adjacent cells are not walls
                if abs(dr) == 1 and abs(dc) == 1:
                    if (self.grid[row + dr, col] != self.WALL and 
                        self.grid[row, col + dc] != self.WALL):
                        neighbors.append((new_row, new_col))
                else:
                    neighbors.append((new_row, new_col))
        
        return neighbors
    
    def get_cost(self, pos1, pos2):
        """Get movement cost (diagonal = 1.414, straight = 1)"""
        if abs(pos1[0] - pos2[0]) == 1 and abs(pos1[1] - pos2[1]) == 1:
            return 1.414  # Diagonal
        return 1.0  # Straight
    
    def visualize(self, title="Grid World", visited_order=None, frontier=None, final_path=None):
        """Visualize the current state of the grid"""
        fig, ax = plt.subplots(figsize=(10, 10))
        
        # Create color map
        colors = {
            self.EMPTY: 'white',
            self.WALL: 'black',
            self.START: 'green',
            self.TARGET: 'red',
            self.VISITED: 'lightblue',
            self.PATH: 'yellow',
            self.FRONTIER: 'orange'
        }
        
        # Draw grid
        display_grid = self.grid.copy()
        
        # Mark visited cells
        if visited_order:
            for pos in visited_order:
                if display_grid[pos[0], pos[1]] not in [self.START, self.TARGET]:
                    display_grid[pos[0], pos[1]] = self.VISITED
        
        # Mark frontier
        if frontier:
            for pos in frontier:
                if display_grid[pos[0], pos[1]] not in [self.START, self.TARGET, self.VISITED]:
                    display_grid[pos[0], pos[1]] = self.FRONTIER
        
        # Mark final path
        if final_path:
            for pos in final_path:
                if display_grid[pos[0], pos[1]] not in [self.START, self.TARGET]:
                    display_grid[pos[0], pos[1]] = self.PATH
        
        # Draw cells
        for i in range(self.rows):
            for j in range(self.cols):
                color = colors[display_grid[i, j]]
                rect = Rectangle((j, self.rows - 1 - i), 1, 1, 
                                facecolor=color, edgecolor='gray', linewidth=0.5)
                ax.add_patch(rect)
        
        ax.set_xlim(0, self.cols)
        ax.set_ylim(0, self.rows)
        ax.set_aspect('equal')
        ax.set_title(title, fontsize=16, fontweight='bold')
        ax.axis('off')
        
        plt.tight_layout()
        plt.show()
        
    def animate_search(self, visited_states, final_path, title, delay=0.1):
        """Animate the search process in real-time"""
        for i, state in enumerate(visited_states):
            clear_output(wait=True)
            visited = state.get('visited', [])
            frontier = state.get('frontier', [])
            
            step_title = f"{title} - Step {i+1}/{len(visited_states)}"
            if i == len(visited_states) - 1 and final_path:
                step_title += " - Path Found!"
            
            self.visualize(step_title, visited, frontier, 
                          final_path if i == len(visited_states) - 1 else None)
            time.sleep(delay)

# Create grid world instance
grid_world = GridWorld(rows=15, cols=15)
print(f"Grid created: {grid_world.rows}x{grid_world.cols}")
print(f"Start: {grid_world.start}, Target: {grid_world.target}")
grid_world.visualize("Initial Grid")

## 1. Breadth-First Search (BFS)

In [ ]:
def bfs_search(grid_world, animate=True):
    """
    Breadth-First Search with real-time visualization
    """
    start = grid_world.start
    target = grid_world.target
    
    # Initialize
    queue = deque([start])
    visited = set([start])
    parent = {start: None}
    visited_order = []
    states = []  # For animation
    
    print("BFS Search Started...")
    
    while queue:
        current = queue.popleft()
        visited_order.append(current)
        
        # Store state for animation
        if animate:
            states.append({
                'visited': visited_order.copy(),
                'frontier': list(queue)
            })
        
        # Check if goal reached
        if current == target:
            print(f"Goal reached! Visited {len(visited_order)} nodes")
            
            # Reconstruct path
            path = []
            node = target
            while node is not None:
                path.append(node)
                node = parent[node]
            path.reverse()
            
            # Calculate cost
            cost = sum(grid_world.get_cost(path[i], path[i+1]) 
                      for i in range(len(path)-1))
            
            print(f"Path length: {len(path)}, Total cost: {cost:.2f}")
            
            if animate:
                grid_world.animate_search(states, path, "Breadth-First Search (BFS)")
            
            return path, cost, len(visited_order)
        
        # Explore neighbors in strict order
        for neighbor in grid_world.get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                parent[neighbor] = current
                queue.append(neighbor)
    
    print("No path found!")
    return None, float('inf'), len(visited_order)

# Run BFS
grid_world.reset_grid()
bfs_path, bfs_cost, bfs_nodes = bfs_search(grid_world, animate=True)

## 2. Depth-First Search (DFS)

In [ ]:
def dfs_search(grid_world, animate=True):
    """
    Depth-First Search with real-time visualization
    """
    start = grid_world.start
    target = grid_world.target
    
    # Initialize
    stack = [start]
    visited = set([start])
    parent = {start: None}
    visited_order = []
    states = []  # For animation
    
    print("DFS Search Started...")
    
    while stack:
        current = stack.pop()
        
        if current in visited_order:
            continue
            
        visited_order.append(current)
        
        # Store state for animation
        if animate:
            states.append({
                'visited': visited_order.copy(),
                'frontier': stack.copy()
            })
        
        # Check if goal reached
        if current == target:
            print(f"Goal reached! Visited {len(visited_order)} nodes")
            
            # Reconstruct path
            path = []
            node = target
            while node is not None:
                path.append(node)
                node = parent[node]
            path.reverse()
            
            # Calculate cost
            cost = sum(grid_world.get_cost(path[i], path[i+1]) 
                      for i in range(len(path)-1))
            
            print(f"Path length: {len(path)}, Total cost: {cost:.2f}")
            
            if animate:
                grid_world.animate_search(states, path, "Depth-First Search (DFS)")
            
            return path, cost, len(visited_order)
        
        # Explore neighbors in REVERSE order for DFS (to maintain clockwise when popping)
        neighbors = grid_world.get_neighbors(current)
        for neighbor in reversed(neighbors):
            if neighbor not in visited:
                visited.add(neighbor)
                parent[neighbor] = current
                stack.append(neighbor)
    
    print("No path found!")
    return None, float('inf'), len(visited_order)

# Run DFS
grid_world.reset_grid()
dfs_path, dfs_cost, dfs_nodes = dfs_search(grid_world, animate=True)

## 3. Uniform Cost Search (UCS)

In [ ]:
def ucs_search(grid_world, animate=True):
    """
    Uniform Cost Search with real-time visualization
    """
    start = grid_world.start
    target = grid_world.target
    
    # Initialize priority queue: (cost, node)
    pq = [(0, start)]
    visited = set()
    parent = {start: None}
    cost_so_far = {start: 0}
    visited_order = []
    states = []  # For animation
    
    print("UCS Search Started...")
    
    while pq:
        current_cost, current = heapq.heappop(pq)
        
        if current in visited:
            continue
        
        visited.add(current)
        visited_order.append(current)
        
        # Store state for animation
        if animate:
            frontier_nodes = [node for _, node in pq]
            states.append({
                'visited': visited_order.copy(),
                'frontier': frontier_nodes
            })
        
        # Check if goal reached
        if current == target:
            print(f"Goal reached! Visited {len(visited_order)} nodes")
            
            # Reconstruct path
            path = []
            node = target
            while node is not None:
                path.append(node)
                node = parent[node]
            path.reverse()
            
            print(f"Path length: {len(path)}, Total cost: {current_cost:.2f}")
            
            if animate:
                grid_world.animate_search(states, path, "Uniform Cost Search (UCS)")
            
            return path, current_cost, len(visited_order)
        
        # Explore neighbors
        for neighbor in grid_world.get_neighbors(current):
            move_cost = grid_world.get_cost(current, neighbor)
            new_cost = current_cost + move_cost
            
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                parent[neighbor] = current
                heapq.heappush(pq, (new_cost, neighbor))
    
    print("No path found!")
    return None, float('inf'), len(visited_order)

# Run UCS
grid_world.reset_grid()
ucs_path, ucs_cost, ucs_nodes = ucs_search(grid_world, animate=True)

## 4. Depth-Limited Search (DLS)

In [ ]:
def dls_search(grid_world, depth_limit=10, animate=True):
    """
    Depth-Limited Search with real-time visualization
    """
    start = grid_world.start
    target = grid_world.target
    
    visited_order = []
    states = []
    
    def dls_recursive(node, depth, visited, parent, path):
        """Recursive DLS helper"""
        nonlocal visited_order, states
        
        visited_order.append(node)
        visited.add(node)
        path.append(node)
        
        # Store state for animation
        if animate:
            states.append({
                'visited': visited_order.copy(),
                'frontier': path.copy()
            })
        
        # Check if goal reached
        if node == target:
            return path.copy()
        
        # Check depth limit
        if depth == 0:
            path.pop()
            return None
        
        # Explore neighbors
        for neighbor in grid_world.get_neighbors(node):
            if neighbor not in visited:
                result = dls_recursive(neighbor, depth - 1, visited, parent, path)
                if result is not None:
                    return result
        
        path.pop()
        return None
    
    print(f"DLS Search Started with depth limit {depth_limit}...")
    
    visited = set()
    parent = {}
    path = []
    
    result_path = dls_recursive(start, depth_limit, visited, parent, path)
    
    if result_path:
        print(f"Goal reached! Visited {len(visited_order)} nodes")
        
        # Calculate cost
        cost = sum(grid_world.get_cost(result_path[i], result_path[i+1]) 
                  for i in range(len(result_path)-1))
        
        print(f"Path length: {len(result_path)}, Total cost: {cost:.2f}")
        
        if animate:
            grid_world.animate_search(states, result_path, 
                                     f"Depth-Limited Search (DLS, limit={depth_limit})")
        
        return result_path, cost, len(visited_order)
    else:
        print(f"No path found within depth limit {depth_limit}!")
        return None, float('inf'), len(visited_order)

# Run DLS with different depth limits
grid_world.reset_grid()
dls_path, dls_cost, dls_nodes = dls_search(grid_world, depth_limit=15, animate=True)

## 5. Iterative Deepening Depth-First Search (IDDFS)

In [ ]:
def iddfs_search(grid_world, max_depth=30, animate=True):
    """
    Iterative Deepening Depth-First Search with real-time visualization
    """
    start = grid_world.start
    target = grid_world.target
    
    all_visited_order = []
    all_states = []
    
    def dls_for_iddfs(node, depth, visited, path):
        """DLS helper for IDDFS"""
        nonlocal all_visited_order, all_states
        
        all_visited_order.append(node)
        visited.add(node)
        path.append(node)
        
        # Store state for animation
        if animate:
            all_states.append({
                'visited': all_visited_order.copy(),
                'frontier': path.copy()
            })
        
        # Check if goal reached
        if node == target:
            return path.copy()
        
        # Check depth limit
        if depth == 0:
            path.pop()
            return None
        
        # Explore neighbors
        for neighbor in grid_world.get_neighbors(node):
            if neighbor not in visited:
                result = dls_for_iddfs(neighbor, depth - 1, visited, path)
                if result is not None:
                    return result
        
        path.pop()
        return None
    
    print("IDDFS Search Started...")
    
    for depth in range(max_depth + 1):
        print(f"  Trying depth {depth}...")
        visited = set()
        path = []
        
        result_path = dls_for_iddfs(start, depth, visited, path)
        
        if result_path:
            print(f"\nGoal reached at depth {depth}! Visited {len(all_visited_order)} nodes total")
            
            # Calculate cost
            cost = sum(grid_world.get_cost(result_path[i], result_path[i+1]) 
                      for i in range(len(result_path)-1))
            
            print(f"Path length: {len(result_path)}, Total cost: {cost:.2f}")
            
            if animate:
                grid_world.animate_search(all_states, result_path, 
                                         f"Iterative Deepening DFS (IDDFS)")
            
            return result_path, cost, len(all_visited_order)
    
    print(f"No path found within max depth {max_depth}!")
    return None, float('inf'), len(all_visited_order)

# Run IDDFS
grid_world.reset_grid()
iddfs_path, iddfs_cost, iddfs_nodes = iddfs_search(grid_world, max_depth=30, animate=True)

## 6. Bidirectional Search

In [ ]:
def bidirectional_search(grid_world, animate=True):
    """
    Bidirectional Search with real-time visualization
    Search simultaneously from start and target
    """
    start = grid_world.start
    target = grid_world.target
    
    # Initialize forward search (from start)
    forward_queue = deque([start])
    forward_visited = {start}
    forward_parent = {start: None}
    
    # Initialize backward search (from target)
    backward_queue = deque([target])
    backward_visited = {target}
    backward_parent = {target: None}
    
    visited_order = []
    states = []
    
    print("Bidirectional Search Started...")
    
    while forward_queue and backward_queue:
        # Expand from forward direction
        if forward_queue:
            current = forward_queue.popleft()
            visited_order.append(current)
            
            # Check if we've met the backward search
            if current in backward_visited:
                print(f"Paths met! Visited {len(visited_order)} nodes")
                
                # Reconstruct path from start to meeting point
                forward_path = []
                node = current
                while node is not None:
                    forward_path.append(node)
                    node = forward_parent[node]
                forward_path.reverse()
                
                # Reconstruct path from meeting point to target
                backward_path = []
                node = backward_parent[current]
                while node is not None:
                    backward_path.append(node)
                    node = backward_parent[node]
                
                # Combine paths
                path = forward_path + backward_path
                
                # Calculate cost
                cost = sum(grid_world.get_cost(path[i], path[i+1]) 
                          for i in range(len(path)-1))
                
                print(f"Path length: {len(path)}, Total cost: {cost:.2f}")
                
                if animate:
                    grid_world.animate_search(states, path, "Bidirectional Search")
                
                return path, cost, len(visited_order)
            
            # Expand neighbors
            for neighbor in grid_world.get_neighbors(current):
                if neighbor not in forward_visited:
                    forward_visited.add(neighbor)
                    forward_parent[neighbor] = current
                    forward_queue.append(neighbor)
        
        # Store state for animation
        if animate:
            states.append({
                'visited': visited_order.copy(),
                'frontier': list(forward_queue) + list(backward_queue)
            })
        
        # Expand from backward direction
        if backward_queue:
            current = backward_queue.popleft()
            visited_order.append(current)
            
            # Check if we've met the forward search
            if current in forward_visited:
                print(f"Paths met! Visited {len(visited_order)} nodes")
                
                # Reconstruct path from start to meeting point
                forward_path = []
                node = current
                while node is not None:
                    forward_path.append(node)
                    node = forward_parent[node]
                forward_path.reverse()
                
                # Reconstruct path from meeting point to target
                backward_path = []
                node = backward_parent[current]
                while node is not None:
                    backward_path.append(node)
                    node = backward_parent[node]
                
                # Combine paths
                path = forward_path + backward_path
                
                # Calculate cost
                cost = sum(grid_world.get_cost(path[i], path[i+1]) 
                          for i in range(len(path)-1))
                
                print(f"Path length: {len(path)}, Total cost: {cost:.2f}")
                
                if animate:
                    grid_world.animate_search(states, path, "Bidirectional Search")
                
                return path, cost, len(visited_order)
            
            # Expand neighbors
            for neighbor in grid_world.get_neighbors(current):
                if neighbor not in backward_visited:
                    backward_visited.add(neighbor)
                    backward_parent[neighbor] = current
                    backward_queue.append(neighbor)
        
        # Store state for animation
        if animate:
            states.append({
                'visited': visited_order.copy(),
                'frontier': list(forward_queue) + list(backward_queue)
            })
    
    print("No path found!")
    return None, float('inf'), len(visited_order)

# Run Bidirectional Search
grid_world.reset_grid()
bidir_path, bidir_cost, bidir_nodes = bidirectional_search(grid_world, animate=True)

## Comparison of All Algorithms

In [ ]:
# Collect results
results = [
    {'name': 'Breadth-First Search (BFS)', 'path': bfs_path, 'cost': bfs_cost, 'nodes': bfs_nodes},
    {'name': 'Depth-First Search (DFS)', 'path': dfs_path, 'cost': dfs_cost, 'nodes': dfs_nodes},
    {'name': 'Uniform Cost Search (UCS)', 'path': ucs_path, 'cost': ucs_cost, 'nodes': ucs_nodes},
    {'name': 'Depth-Limited Search (DLS)', 'path': dls_path, 'cost': dls_cost, 'nodes': dls_nodes},
    {'name': 'Iterative Deepening DFS (IDDFS)', 'path': iddfs_path, 'cost': iddfs_cost, 'nodes': iddfs_nodes},
    {'name': 'Bidirectional Search', 'path': bidir_path, 'cost': bidir_cost, 'nodes': bidir_nodes}
]

# Sort by cost
results_sorted = sorted([r for r in results if r['path'] is not None], key=lambda x: x['cost'])

print("\n" + "="*80)
print("COMPARISON OF SEARCH ALGORITHMS")
print("="*80)
print(f"\n{'Rank':<6} {'Algorithm':<40} {'Cost':<12} {'Path Len':<12} {'Nodes Visited':<15}")
print("-"*80)

for i, result in enumerate(results_sorted, 1):
    path_len = len(result['path']) if result['path'] else 0
    print(f"{i:<6} {result['name']:<40} {result['cost']:<12.2f} {path_len:<12} {result['nodes']:<15}")

print("\n" + "="*80)
print("CONCLUSIONS:")
print("="*80)
print("\n1. Uniform Cost Search (UCS):")
print("   - Optimal: Guarantees shortest path based on cost")
print("   - Complete: Will find solution if one exists")
print("   - Best for weighted graphs with variable edge costs\n")

print("2. Breadth-First Search (BFS):")
print("   - Optimal for uniform cost edges")
print("   - Explores layer by layer")
print("   - Good space complexity for shallow solutions\n")

print("3. Bidirectional Search:")
print("   - Can reduce search space significantly")
print("   - Searches from both start and goal simultaneously")
print("   - Efficient when both directions are possible\n")

print("4. Iterative Deepening DFS (IDDFS):")
print("   - Combines benefits of BFS and DFS")
print("   - Better space complexity than BFS")
print("   - Optimal for uniform costs\n")

print("5. Depth-First Search (DFS):")
print("   - Not optimal: May not find shortest path")
print("   - Good space complexity")
print("   - Can get stuck in deep branches\n")

print("6. Depth-Limited Search (DLS):")
print("   - Incomplete: May not find solution if beyond depth limit")
print("   - Useful when depth of solution is known")
print("   - Prevents infinite loops in infinite spaces\n")

print("="*80)

## Visualize All Paths Together

In [ ]:
# Create a comparison visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

algorithms = [
    ('BFS', bfs_path, bfs_cost, bfs_nodes),
    ('DFS', dfs_path, dfs_cost, dfs_nodes),
    ('UCS', ucs_path, ucs_cost, ucs_nodes),
    ('DLS', dls_path, dls_cost, dls_nodes),
    ('IDDFS', iddfs_path, iddfs_cost, iddfs_nodes),
    ('Bidirectional', bidir_path, bidir_cost, bidir_nodes)
]

colors = {
    grid_world.EMPTY: 'white',
    grid_world.WALL: 'black',
    grid_world.START: 'green',
    grid_world.TARGET: 'red',
    grid_world.PATH: 'yellow'
}

for idx, (name, path, cost, nodes) in enumerate(algorithms):
    ax = axes[idx]
    
    # Create display grid
    display_grid = grid_world.grid.copy()
    
    if path:
        # Mark path
        for pos in path:
            if display_grid[pos[0], pos[1]] not in [grid_world.START, grid_world.TARGET]:
                display_grid[pos[0], pos[1]] = grid_world.PATH
    
    # Draw grid
    for i in range(grid_world.rows):
        for j in range(grid_world.cols):
            color = colors[display_grid[i, j]]
            rect = Rectangle((j, grid_world.rows - 1 - i), 1, 1,
                           facecolor=color, edgecolor='gray', linewidth=0.5)
            ax.add_patch(rect)
    
    ax.set_xlim(0, grid_world.cols)
    ax.set_ylim(0, grid_world.rows)
    ax.set_aspect('equal')
    
    if path:
        ax.set_title(f"{name}\nCost: {cost:.2f}, Path: {len(path)}, Nodes: {nodes}", 
                    fontweight='bold', fontsize=10)
    else:
        ax.set_title(f"{name}\nNo path found", fontweight='bold', fontsize=10)
    
    ax.axis('off')

plt.tight_layout()
plt.suptitle('Comparison of All Search Algorithms', fontsize=16, fontweight='bold', y=1.00)
plt.show()

## Custom Grid Configuration
You can modify the grid, add/remove walls, and change start/target positions:

In [ ]:
# Example: Create a custom grid
custom_grid = GridWorld(rows=20, cols=20)

# Change start and target
custom_grid.start = (2, 2)
custom_grid.target = (17, 17)

# Add custom walls (create a maze)
custom_grid.reset_grid()

# Add maze walls
for i in range(5, 15):
    custom_grid.grid[i, 8] = custom_grid.WALL
    custom_grid.grid[i, 12] = custom_grid.WALL

for j in range(5, 10):
    custom_grid.grid[7, j] = custom_grid.WALL
    custom_grid.grid[12, j] = custom_grid.WALL

# Update start and target on grid
custom_grid.grid[custom_grid.start[0], custom_grid.start[1]] = custom_grid.START
custom_grid.grid[custom_grid.target[0], custom_grid.target[1]] = custom_grid.TARGET

# Visualize custom grid
custom_grid.visualize("Custom Grid Configuration")

# Run BFS on custom grid
print("\nRunning BFS on custom grid...")
custom_path, custom_cost, custom_nodes = bfs_search(custom_grid, animate=True)